In [9]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input, concatenate, add
from tensorflow.keras.layers import Conv2D, MaxPooling2D,MaxPool2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD, RMSprop

import random
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import VGG16, ResNet50, VGG16

import os
import itertools
import cv2
from sklearn.metrics import log_loss, confusion_matrix
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os
import warnings
warnings.simplefilter("ignore", UserWarning)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_width = 224
image_height = 224

image_size = (image_width, image_height)

data_datagen = ImageDataGenerator(rescale=1.0/255,  validation_split=0.20, featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=True,  
        vertical_flip=False)

train_generator = data_datagen.flow_from_directory(
        '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/train',  
        target_size=image_size,  
        batch_size=64,
        class_mode='binary',
         subset='training')

validation_generator = data_datagen.flow_from_directory(
        '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/val',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=64,
        class_mode='binary',
         subset='validation', shuffle = False)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
       '../input/train-val-test-tcga-coad-msi-mss/tcga_coad_msi_mss/test',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=64,
        class_mode='binary',shuffle = False)

In [11]:
def create_model():
    resnet50 = ResNet50(include_top=False, input_shape=(224, 224, 3))
    
    model = Sequential()
    model.add(resnet50)
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dense(1, activation='sigmoid'))   
    model.summary()
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['acc'])
    return model
    

In [12]:
model = create_model()

In [14]:
n_train = 123080
batch_size = 64
n_valid = 19233
history = model.fit_generator(
       train_generator,
       steps_per_epoch=n_train//batch_size,
       epochs=10,
       validation_data=test_generator,
       validation_steps=n_valid//batch_size)

In [15]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend(loc='best')

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc='best');


In [46]:
n_test = 19233
Y_pred = model.predict_generator(test_generator, n_test//64+1)
y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)

In [17]:
target_names=['MSI', 'MSS']

In [36]:
from sklearn.metrics import classification_report

In [53]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [58]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

    model.save("model.h5")
print("Saved model to disk")

In [55]:
from tensorflow.keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [59]:
loaded_model.summary()

In [39]:
print(classification_report (test_generator.classes, y_pred, target_names = target_names))